In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
slices = 5

In [3]:
def get_all_result(model, part_type, datasets, slices, result, retrain = False):
    all_files = os.listdir(os.getcwd() + '/Result')

    file_model = './Result/' + model
    # print(os.path.abspath(file_model))
    file_model_dir = os.listdir(os.path.abspath(file_model))
    # interaction
    file_model_part = f'part_type_{part_type}'

    # print(file_model_part)
    file_model_part_dir = os.listdir(os.path.abspath(file_model + '/' + file_model_part))

    # dataset
    if datasets == 'rt':
        file_model_part_dataset = 'rt'
    else:
        file_model_part_dataset = 'tp'
    file_model_part_dataset_dir = os.listdir(os.path.abspath(file_model + '/' + file_model_part + '/' + file_model_part_dataset))


    # slices
    file_model_part_dataset_result = None
    for item in file_model_part_dataset_dir:
        # print(item)
        if 'slices_' + str(slices) == item:
            file_model_part_dataset_result = item
    # print('slices_' + str(slices))
    # print(file_model_part_dataset_result)
    file_model_part_dataset_result_dir = os.listdir(os.path.abspath(file_model + '/' + file_model_part + '/' + file_model_part_dataset + '/' + file_model_part_dataset_result))

    # metrics
    if result == 'metrics':
        file_model_part_dataset_result_metrics = file_model_part_dataset_result_dir[0]
    else:
        file_model_part_dataset_result_metrics = file_model_part_dataset_result_dir[1]

    file_model_part_dataset_result_metrics_dir = os.listdir(os.path.abspath(file_model + '/' + file_model_part + '/' + file_model_part_dataset + '/' + file_model_part_dataset_result + '/' + file_model_part_dataset_result_metrics))

    all_metrics = []
    # 文件排序
    file_model_part_dataset_result_metrics_dir = sorted(file_model_part_dataset_result_metrics_dir)

    for item in file_model_part_dataset_result_metrics_dir:
        if item[-1] == 'v':
            temp = os.path.abspath(file_model + '/' + file_model_part + '/' + file_model_part_dataset + '/' + file_model_part_dataset_result + '/' + file_model_part_dataset_result_metrics) + '/' + item
            final_file = ''
            for i in range(len(temp)):
                if temp[i] == '\\':
                    final_file += '/'
                else:
                    final_file += temp[i]
            # print(final_file)
            df = pd.read_csv(final_file).to_numpy()

            if df.shape[0] != 0:
                if result == 'metrics':
                    all_metrics.append(list(df[-1]))
                elif retrain:
                    Sum = 0.
                    cnt = 0
                    for i in range(len(df)):
                            Sum += df[i][2]
                            cnt += 1
                    if cnt == 0:
                        cnt = 1
                    all_metrics.append([1, '_', Sum / cnt])
                else:
                    Sum = 0.
                    cnt = 0
                    for i in range(len(df)):
                        if df[i][1] == 'Aggregators':
                            Sum += df[i][2]
                            cnt += 1
                    if cnt == 0:
                        cnt = 1
                    all_metrics.append([1, '_', Sum / cnt])
            else: 
                print(final_file, '实验尚未进行')
                break

            print(final_file)

    print('----------------------------------------------------------------------------------------------------------------------------------')
    all_metrics = np.array(all_metrics)
    # print(all_metrics)
    if len(all_metrics) != 0 and result == 'time':
        all_metrics = all_metrics[:, 1:]
    elif len(all_metrics) != 0 and result == 'metrics':
        all_metrics = all_metrics[:, 1:]

    return all_metrics

def makedir(path):
    path = path.strip()
    path = path.rstrip("\\")
    isExists = os.path.exists(path)
    if not isExists:
        os.makedirs(path)
        return True
    return False

def show(part_type):
    if part_type == 1:
        print('node2vec experiment!')
    elif part_type == 2:
        print('SISA2 experiment!')
    else:
        print('Pure SISA experiment!')


def get_result(model, part_type, datasets, slices, result, retrain = False):
    ans = []
    file_name = None
    flag = True
    try:
        if result == 'metrics':
            ans.append(get_all_result(model, part_type, datasets, slices, result, retrain))
            ans = pd.DataFrame(np.array(ans).reshape(-1, 5))
            ans.columns = ['MAE','RMSE', 'NMAE', 'MRE', 'NPRE']
            makedir(f'./Result/{str(datasets)}_result/')
            file_name = f'./Result/{str(datasets)}_result/' + str(model) + '_' + str(part_type) + '_' + str(slices) + '_' + str(result) + '.xlsx'
        elif result == 'time':
            ans = get_all_result(model, part_type, datasets, slices, result, retrain)[:, 1:]
            ans = pd.DataFrame(np.array(ans))
            ans.columns = ['agg_time']
            makedir(f'./Result/{str(datasets)}_result/')
            file_name = f'./Result/{str(datasets)}_result/' + str(model) + '_' + str(part_type) + '_' + str(slices) + '_' + str(result) + '.xlsx'
        

        if result == 'all':

            # metrics
            ans.append(get_all_result(model, part_type, datasets, slices, 'metrics', retrain))
            # ans = pd.DataFrame(np.array(ans).reshape(-1, 5))
            ans = np.array(ans).reshape(-1, 5)

            # time
            time_result = np.zeros((ans.shape[0], 1))

            if ans.shape[0] != 0:
                temp = np.array(get_all_result(model, part_type, datasets, slices, 'time', retrain)[:, 1:])
                # print(ans.shape)
                # print(time_result, temp)
                # print(temp.shape, time_result.shape)
                for i in range(ans.shape[0]):
                    time_result[i] = temp[i]

            # print(ans.shape, time_result.shape)
            ans = np.concatenate([ans, time_result], axis = 1)

            # 填充
            a = np.zeros((5, 6))
            r, c = ans.nonzero()
            a[r, c] = ans[r, c]
            ans = a

            # print(ans)
            # print(np.array(get_all_result(model, part_type, datasets, slices, 'time')[:, 1:]))
            ans = pd.DataFrame(np.array(ans))

            if retrain:
                ans.columns = ['MAE','RMSE', 'NMAE', 'MRE', 'NPRE', 'retrain_time']
            else:
                ans.columns = ['MAE','RMSE', 'NMAE', 'MRE', 'NPRE', 'agg_time']

            makedir(f'./Result/{str(datasets)}_result/')
            file_name = f'./Result/{str(datasets)}_result/' + str(model) + '_' + str(part_type) + '_' + str(slices) + '_' + str(result) + '.xlsx'

    except IOError:
        show(part_type)
        print('实验尚未进行')
        flag = False

    if flag:
        show(part_type)
    
    if np.array(ans).shape[0] != 5 and flag:
        print('实验未完成')
    
    if type(ans) != list:
        ans.to_excel(file_name)
        return ans
    else:
        return None

# Retrain

In [4]:
get_result('CSMF', 10, 'rt', 1, 'all', True)

/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/rt/slices_1/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/rt/slices_1/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/rt/slices_1/time/Fin

,MAE,RMSE,NMAE,MRE,NPRE,retrain_time
0,0.381232,1.280184,0.419521,0.233479,0.905320,58.291479
1,0.352104,1.237005,0.387357,0.203147,0.822566,96.909429
2,0.332131,1.204398,0.365400,0.185477,0.779149,152.976478
3,0.321007,1.182212,0.353164,0.177721,0.742926,225.829644
4,0.311297,1.151459,0.342702,0.173414,0.730462,375.279533


In [5]:
get_result('NeuCF', 10, 'rt', 1, 'all', True)

/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/

,MAE,RMSE,NMAE,MRE,NPRE,retrain_time
0,0.392078,1.313922,0.431424,0.294174,1.076311,112.934530
1,0.363028,1.273544,0.399488,0.256225,0.944757,137.566726
2,0.346025,1.237055,0.380859,0.236537,0.893256,186.405459
3,0.333460,1.212016,0.366833,0.223961,0.874355,162.086323
4,0.322231,1.176460,0.354643,0.216670,0.858448,185.993939


In [6]:
get_result('GraphMF', 10, 'rt', 1, 'all', True)

/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/rt/slices_1/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/rt/slices_1/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type

,MAE,RMSE,NMAE,MRE,NPRE,retrain_time
0,0.321518,1.243862,0.353885,0.145110,0.701554,132.858337
1,0.306773,1.214771,0.337634,0.134021,0.672278,285.520636
2,0.299721,1.201695,0.329798,0.128847,0.658659,409.647457
3,0.293260,1.187610,0.322822,0.123628,0.641647,565.838042
4,0.288732,1.175406,0.317914,0.120914,0.632435,618.144649


In [7]:
get_result('MF', 10, 'rt', 1, 'all', True)

/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/rt/slices_1/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/rt/slices_1/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/rt/slices_1/time/Final_time_density_

,MAE,RMSE,NMAE,MRE,NPRE,retrain_time
0,0.421913,1.324964,0.464470,0.324257,1.406977,61.145276
1,0.371186,1.271622,0.408538,0.278485,1.124416,65.037148
2,0.354635,1.254018,0.390518,0.263321,1.031402,68.935656
3,0.345227,1.230065,0.379931,0.254233,1.008424,74.471885
4,0.338208,1.201685,0.372070,0.255180,1.028326,96.465300


In [8]:
get_result('CSMF', 10, 'tp', 1, 'all', True)

/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/tp/slices_1/metrics/Final_result_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/tp/slices_1/metrics/Final_result_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/tp/slices_1/metrics/Final_result_density_tp_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/tp/slices_1/metrics/Final_result_density_tp_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/tp/slices_1/metrics/Final_result_density_tp_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/tp/slices_1/time/Final_time_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/tp/slices_1/time/Final_time_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/tp/slices_1/time/Fin

,MAE,RMSE,NMAE,MRE,NPRE,retrain_time
0,13.971522,45.195302,0.293879,0.227560,1.048403,94.455542
1,12.828636,42.110755,0.269765,0.206441,0.939619,154.747569
2,11.991392,40.105389,0.252061,0.191275,0.865923,206.394382
3,11.371379,38.523807,0.239113,0.178607,0.821420,254.214706
4,11.041444,37.399120,0.232154,0.173307,0.828952,352.410429


In [9]:
get_result('NeuCF', 10, 'tp', 1, 'all', True)

/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/tp/slices_1/metrics/Final_result_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/tp/slices_1/metrics/Final_result_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/tp/slices_1/metrics/Final_result_density_tp_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/tp/slices_1/metrics/Final_result_density_tp_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/tp/slices_1/metrics/Final_result_density_tp_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/tp/slices_1/time/Final_time_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/tp/slices_1/time/Final_time_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/tp/slices_1/

,MAE,RMSE,NMAE,MRE,NPRE,retrain_time
0,16.512076,50.636437,0.347242,0.314250,1.642096,81.843594
1,15.105738,46.872090,0.317647,0.293152,1.574311,115.067800
2,13.949186,43.662707,0.293232,0.275080,1.489424,144.980059
3,13.243787,41.599470,0.278368,0.266356,1.404215,195.048683
4,12.449648,39.647153,0.261662,0.245847,1.359830,250.932269


In [10]:
get_result('GraphMF', 10, 'tp', 1, 'all', True)

/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/tp/slices_1/metrics/Final_result_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/tp/slices_1/metrics/Final_result_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/tp/slices_1/metrics/Final_result_density_tp_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/tp/slices_1/metrics/Final_result_density_tp_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/tp/slices_1/metrics/Final_result_density_tp_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/tp/slices_1/time/Final_time_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/tp/slices_1/time/Final_time_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type

,MAE,RMSE,NMAE,MRE,NPRE,retrain_time
0,11.740687,42.796563,0.246875,0.158028,0.822926,258.631380
1,10.848979,40.212610,0.228156,0.144228,0.774868,403.274969
2,10.314256,38.327734,0.216842,0.135466,0.745669,516.319347
3,9.958744,37.293482,0.209371,0.129499,0.727358,645.027591
4,9.760403,36.593136,0.205370,0.126653,0.720668,799.489423


In [11]:
get_result('MF', 10, 'tp', 1, 'all', True)

/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/tp/slices_1/metrics/Final_result_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/tp/slices_1/metrics/Final_result_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/tp/slices_1/metrics/Final_result_density_tp_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/tp/slices_1/metrics/Final_result_density_tp_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/tp/slices_1/metrics/Final_result_density_tp_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/tp/slices_1/time/Final_time_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/tp/slices_1/time/Final_time_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/tp/slices_1/time/Final_time_density_

,MAE,RMSE,NMAE,MRE,NPRE,retrain_time
0,16.426994,48.348161,0.345584,0.382079,2.686005,67.084632
1,14.493483,44.381298,0.304673,0.335571,2.190485,87.151610
2,13.636238,42.688507,0.286819,0.317318,2.021180,95.329702
3,13.125734,41.530963,0.275956,0.305012,1.889587,101.998270
4,12.894151,40.012797,0.270995,0.313327,1.968129,126.096581


*****

In [12]:
get_result('CSMF', 1, 'rt', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_1/rt/slices_5/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_1/rt/slices_5/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_1/rt/slices_5/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_1/rt/slices_5/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_1/rt/slices_5/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_1/rt/slices_5/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_1/rt/slices_5/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_1/rt/slices_5/time/Final_time_

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,0.553634,1.536654,0.609393,0.466756,2.015448,13.249884
1,0.521528,1.481081,0.573946,0.438093,1.792567,12.698410
2,0.514238,1.468144,0.565938,0.427566,1.770080,16.955495
3,0.492257,1.427981,0.541614,0.405275,1.684665,19.802136
4,0.492931,1.429421,0.542911,0.411019,1.748340,20.502277


In [13]:
get_result('NeuCF', 1, 'rt', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_1/rt/slices_5/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_1/rt/slices_5/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_1/rt/slices_5/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_1/rt/slices_5/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_1/rt/slices_5/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_1/rt/slices_5/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_1/rt/slices_5/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_1/rt/slices_5/time/Fin

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,0.482301,1.414551,0.530998,0.422231,2.056637,8.617153
1,0.447009,1.358422,0.492049,0.387019,1.928124,10.169813
2,0.428050,1.335279,0.471497,0.365729,1.880330,11.420890
3,0.400578,1.303824,0.441038,0.336063,1.494370,14.036674
4,0.386466,1.285129,0.425635,0.313813,1.372867,16.375243


In [14]:
get_result('GraphMF', 1, 'rt', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_1/rt/slices_5/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_1/rt/slices_5/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_1/rt/slices_5/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_1/rt/slices_5/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_1/rt/slices_5/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_1/rt/slices_5/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_1/rt/slices_5/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_1/rt/s

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,0.346552,1.268372,0.381429,0.192267,0.806109,28.935094
1,0.328515,1.237921,0.361548,0.171040,0.761055,35.095517
2,0.318232,1.220306,0.350095,0.160117,0.732280,43.970476
3,0.309600,1.205798,0.340762,0.152289,0.709732,46.916364
4,0.305141,1.191504,0.335751,0.148470,0.696431,55.933195


In [15]:
get_result('MF', 1, 'rt', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_1/rt/slices_5/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_1/rt/slices_5/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_1/rt/slices_5/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_1/rt/slices_5/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_1/rt/slices_5/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_1/rt/slices_5/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_1/rt/slices_5/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_1/rt/slices_5/time/Final_time_density_rt_0.20.

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,0.751829,2.059745,0.827147,0.666908,3.217934,7.368153
1,0.711287,1.935965,0.782927,0.655438,3.013056,12.811377
2,0.695839,1.851640,0.765723,0.679910,3.089464,6.691185
3,0.676129,1.786685,0.744515,0.680527,2.953508,14.837882
4,0.674864,1.788649,0.742799,0.679520,2.952906,19.311756


In [16]:
get_result('CSMF', 1, 'tp', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_1/tp/slices_5/metrics/Final_result_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_1/tp/slices_5/metrics/Final_result_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_1/tp/slices_5/metrics/Final_result_density_tp_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_1/tp/slices_5/metrics/Final_result_density_tp_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_1/tp/slices_5/metrics/Final_result_density_tp_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_1/tp/slices_5/time/Final_time_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_1/tp/slices_5/time/Final_time_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_1/tp/slices_5/time/Final_time_

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,23.513325,65.912155,0.494322,0.443903,2.204515,16.814751
1,22.547432,63.079445,0.474267,0.428868,2.045134,17.188501
2,22.117840,62.456580,0.465074,0.415796,1.974611,18.136719
3,21.868866,62.150837,0.459877,0.410962,1.930406,24.257547
4,21.274975,59.865710,0.447477,0.400549,1.947320,28.743601


In [17]:
get_result('NeuCF', 1, 'tp', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_1/tp/slices_5/metrics/Final_result_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_1/tp/slices_5/metrics/Final_result_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_1/tp/slices_5/metrics/Final_result_density_tp_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_1/tp/slices_5/metrics/Final_result_density_tp_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_1/tp/slices_5/metrics/Final_result_density_tp_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_1/tp/slices_5/time/Final_time_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_1/tp/slices_5/time/Final_time_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_1/tp/slices_5/time/Fin

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,24.349040,66.082970,0.511777,0.493867,2.048697,9.898722
1,20.118294,57.172813,0.422874,0.414025,1.792910,12.864372
2,18.351050,53.570820,0.385838,0.363495,1.761439,14.642346
3,16.689625,50.014603,0.350686,0.328772,1.364373,17.574454
4,15.759127,48.018208,0.331268,0.302936,1.483804,19.926344


In [18]:
get_result('GraphMF', 1, 'tp', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_1/tp/slices_5/metrics/Final_result_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_1/tp/slices_5/metrics/Final_result_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_1/tp/slices_5/metrics/Final_result_density_tp_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_1/tp/slices_5/metrics/Final_result_density_tp_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_1/tp/slices_5/metrics/Final_result_density_tp_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_1/tp/slices_5/time/Final_time_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_1/tp/slices_5/time/Final_time_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_1/tp/s

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,13.676613,46.311800,0.287542,0.211396,0.973130,28.790974
1,12.634592,43.537224,0.265594,0.191254,0.900765,36.348507
2,11.937988,41.581050,0.251002,0.176387,0.869447,40.500073
3,11.394626,40.155052,0.239279,0.164793,0.806804,47.712588
4,10.981550,39.071260,0.230878,0.158100,0.787131,58.214181


In [19]:
get_result('MF', 1, 'tp', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_1/tp/slices_5/metrics/Final_result_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_1/tp/slices_5/metrics/Final_result_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_1/tp/slices_5/metrics/Final_result_density_tp_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_1/tp/slices_5/metrics/Final_result_density_tp_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_1/tp/slices_5/metrics/Final_result_density_tp_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_1/tp/slices_5/time/Final_time_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_1/tp/slices_5/time/Final_time_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_1/tp/slices_5/time/Final_time_density_tp_0.20.

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,41.385647,115.212770,0.870787,0.770237,5.108451,7.013892
1,38.715672,104.183925,0.813932,0.805822,5.172738,9.821281
2,37.959404,100.843119,0.797906,0.816959,5.320587,13.001098
3,36.393730,94.800090,0.765601,0.833413,5.256150,15.875358
4,34.922030,89.640396,0.733897,0.841701,5.043265,20.875837


*****

# Receraser user_based

In [20]:
get_result('CSMF', 2, 'rt', slices, 'all')

SISA2 experiment!
实验尚未进行


In [21]:
get_result('NeuCF', 2, 'rt', slices, 'all')

SISA2 experiment!
实验尚未进行


In [22]:
get_result('GraphMF', 2, 'rt', slices, 'all')

SISA2 experiment!
实验尚未进行


In [23]:
get_result('MF', 2, 'rt', slices, 'all')

SISA2 experiment!
实验尚未进行


In [24]:
get_result('CSMF', 2, 'tp', slices, 'all')

SISA2 experiment!
实验尚未进行


In [25]:
get_result('NeuCF', 2, 'tp', slices, 'all')

SISA2 experiment!
实验尚未进行


In [26]:
get_result('GraphMF', 2, 'tp', slices, 'all')

SISA2 experiment!
实验尚未进行


In [27]:
get_result('MF', 2, 'tp', slices, 'all')

SISA2 experiment!
实验尚未进行


*****

*****

In [28]:
get_result('CSMF', 3, 'rt', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_3/rt/slices_5/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_3/rt/slices_5/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_3/rt/slices_5/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_3/rt/slices_5/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_3/rt/slices_5/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_3/rt/slices_5/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_3/rt/slices_5/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_3/rt/slices_5/time/Final_time_

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,0.524577,1.514881,0.577429,0.427494,1.785409,14.637220
1,0.511020,1.499550,0.562791,0.410606,1.564236,19.591481
2,0.494511,1.464065,0.544128,0.393109,1.683856,20.541901
3,0.476160,1.466198,0.523767,0.351888,1.405009,28.802431
4,0.461915,1.420351,0.508348,0.352899,1.358834,30.564494


In [29]:
get_result('NeuCF', 3, 'rt', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_3/rt/slices_5/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_3/rt/slices_5/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_3/rt/slices_5/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_3/rt/slices_5/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_3/rt/slices_5/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_3/rt/slices_5/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_3/rt/slices_5/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_3/rt/slices_5/time/Fin

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,0.476795,1.406324,0.524875,0.422524,2.290029,10.871573
1,0.445929,1.361434,0.490766,0.392371,2.068318,15.045935
2,0.421950,1.335810,0.464453,0.366345,1.698534,18.911112
3,0.401849,1.311578,0.442376,0.330407,1.443432,20.656188
4,0.393714,1.299820,0.433456,0.321415,1.458720,23.577317


In [30]:
get_result('GraphMF', 3, 'rt', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_3/rt/slices_5/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_3/rt/slices_5/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_3/rt/slices_5/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_3/rt/slices_5/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_3/rt/slices_5/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_3/rt/slices_5/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_3/rt/slices_5/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_3/rt/s

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,0.368496,1.302992,0.405631,0.216638,0.857764,35.845094
1,0.347934,1.267125,0.382844,0.195376,0.823383,43.280086
2,0.338666,1.251424,0.372464,0.185758,0.794572,52.098697
3,0.327587,1.230075,0.360766,0.177659,0.770797,58.399552
4,0.321771,1.216786,0.354260,0.173331,0.764983,68.039703


In [31]:
get_result('MF', 3, 'rt', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_3/rt/slices_5/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_3/rt/slices_5/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_3/rt/slices_5/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_3/rt/slices_5/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_3/rt/slices_5/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_3/rt/slices_5/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_3/rt/slices_5/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_3/rt/slices_5/time/Final_time_density_rt_0.20.

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,0.746751,2.041461,0.821947,0.662575,3.205842,6.925704
1,0.736763,2.022347,0.810518,0.645220,3.152114,12.569344
2,0.727730,1.991984,0.800783,0.644443,3.186949,18.386373
3,0.702522,1.936362,0.773922,0.626161,3.033952,32.237418
4,0.689608,1.894612,0.759038,0.609641,3.024852,29.356486


In [32]:
get_result('CSMF', 3, 'tp', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_3/tp/slices_5/metrics/Final_result_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_3/tp/slices_5/metrics/Final_result_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_3/tp/slices_5/metrics/Final_result_density_tp_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_3/tp/slices_5/metrics/Final_result_density_tp_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_3/tp/slices_5/metrics/Final_result_density_tp_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_3/tp/slices_5/time/Final_time_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_3/tp/slices_5/time/Final_time_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_3/tp/slices_5/time/Final_time_

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,22.771120,65.981490,0.478894,0.420219,2.065907,18.316348
1,22.141546,65.137420,0.465523,0.397525,2.126333,23.559067
2,21.075153,62.116985,0.443159,0.381466,1.882960,27.259416
3,20.459267,62.121166,0.430295,0.345853,1.770762,33.944691
4,19.894140,61.161522,0.418511,0.334072,1.778583,35.683556


In [33]:
get_result('NeuCF', 3, 'tp', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_3/tp/slices_5/metrics/Final_result_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_3/tp/slices_5/metrics/Final_result_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_3/tp/slices_5/metrics/Final_result_density_tp_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_3/tp/slices_5/metrics/Final_result_density_tp_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_3/tp/slices_5/metrics/Final_result_density_tp_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_3/tp/slices_5/time/Final_time_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_3/tp/slices_5/time/Final_time_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_3/tp/slices_5/time/Fin

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,22.936802,63.008324,0.482366,0.476103,2.189929,13.222768
1,19.949474,57.077827,0.419391,0.405912,1.892823,17.071018
2,18.478191,53.875317,0.388579,0.368458,1.603413,20.844618
3,17.519193,51.858793,0.368544,0.343589,1.546406,24.891179
4,17.078354,50.780117,0.359345,0.334213,1.510395,27.988450


In [34]:
get_result('GraphMF', 3, 'tp', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_3/tp/slices_5/metrics/Final_result_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_3/tp/slices_5/metrics/Final_result_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_3/tp/slices_5/metrics/Final_result_density_tp_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_3/tp/slices_5/metrics/Final_result_density_tp_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_3/tp/slices_5/metrics/Final_result_density_tp_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_3/tp/slices_5/time/Final_time_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_3/tp/slices_5/time/Final_time_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_3/tp/s

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,15.089726,49.388600,0.317269,0.248541,1.328462,34.097008
1,13.989352,46.332684,0.294078,0.227593,1.170349,43.665662
2,13.281751,44.129020,0.279309,0.216933,1.128157,50.266024
3,12.813992,42.827880,0.269350,0.209099,1.036159,60.154593
4,12.410585,41.738541,0.261148,0.199668,1.058956,70.081438


In [35]:
get_result('MF', 3, 'tp', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_3/tp/slices_5/metrics/Final_result_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_3/tp/slices_5/metrics/Final_result_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_3/tp/slices_5/metrics/Final_result_density_tp_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_3/tp/slices_5/metrics/Final_result_density_tp_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_3/tp/slices_5/metrics/Final_result_density_tp_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_3/tp/slices_5/time/Final_time_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_3/tp/slices_5/time/Final_time_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_3/tp/slices_5/time/Final_time_density_tp_0.20.

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,41.523693,115.789566,0.872964,0.768829,4.999567,10.743415
1,38.711426,106.533080,0.813904,0.749075,4.980037,18.439094
2,35.515090,98.174722,0.746482,0.701921,4.449243,25.698500
3,33.154380,92.719380,0.697731,0.674245,4.204892,28.948037
4,30.909857,87.398050,0.649703,0.638467,3.840594,39.206599


# Receraser item_based

In [36]:
get_result('CSMF', 4, 'rt', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [37]:
get_result('NeuCF', 4, 'rt', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [38]:
get_result('GraphMF', 4, 'rt', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [39]:
get_result('MF', 4, 'rt', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [40]:
get_result('CSMF', 4, 'tp', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [41]:
get_result('NeuCF', 4, 'tp', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [42]:
get_result('GraphMF', 4, 'tp', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [43]:
get_result('MF', 4, 'tp', slices, 'all')

Pure SISA experiment!
实验尚未进行


*****

*****

# SISA2

In [44]:
get_result('CSMF', 5, 'rt', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_5/rt/slices_5/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_5/rt/slices_5/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_5/rt/slices_5/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_5/rt/slices_5/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_5/rt/slices_5/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_5/rt/slices_5/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_5/rt/slices_5/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_5/rt/slices_5/time/Final_time_

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,0.440785,1.368685,0.485055,0.377255,1.667402,26.087243
1,0.410742,1.329196,0.451846,0.332981,1.411446,32.583205
2,0.403900,1.319289,0.444642,0.323004,1.245797,34.443926
3,0.390911,1.299281,0.430160,0.311772,1.239224,40.851733
4,0.393640,1.305261,0.432998,0.304997,1.192203,47.240753


In [45]:
get_result('NeuCF', 5, 'rt', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_5/rt/slices_5/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_5/rt/slices_5/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_5/rt/slices_5/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_5/rt/slices_5/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_5/rt/slices_5/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_5/rt/slices_5/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_5/rt/slices_5/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_5/rt/slices_5/time/Fin

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,0.396871,1.297349,0.436891,0.316830,1.407246,24.155561
1,0.369182,1.279188,0.406377,0.262081,0.979024,34.648639
2,0.354376,1.262925,0.390112,0.244602,0.928043,44.247143
3,0.344369,1.243163,0.378887,0.232738,0.886130,57.001077
4,0.332609,1.223811,0.366236,0.213287,0.849169,70.028702


In [46]:
get_result('GraphMF', 5, 'rt', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_5/rt/slices_5/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_5/rt/slices_5/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_5/rt/slices_5/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_5/rt/slices_5/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_5/rt/slices_5/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_5/rt/slices_5/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_5/rt/slices_5/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_5/rt/s

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,0.341432,1.256147,0.375890,0.186315,0.817112,66.929804
1,0.324625,1.224452,0.357349,0.170205,0.763682,88.570649
2,0.312838,1.196490,0.344489,0.162211,0.755691,106.052637
3,0.306653,1.185328,0.337672,0.159213,0.743484,126.284805
4,0.301734,1.176427,0.332080,0.152734,0.708688,156.032411


In [47]:
get_result('MF', 5, 'rt', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_5/rt/slices_5/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_5/rt/slices_5/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_5/rt/slices_5/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_5/rt/slices_5/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_5/rt/slices_5/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_5/rt/slices_5/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_5/rt/slices_5/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_5/rt/slices_5/time/Final_time_density_rt_0.20.

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,0.568700,1.611065,0.626001,0.506258,2.279673,26.646005
1,0.506450,1.456269,0.557366,0.458719,1.874006,35.687654
2,0.486147,1.421588,0.534890,0.424386,1.572531,48.060033
3,0.451137,1.348686,0.496780,0.401213,1.550106,52.914613
4,0.453907,1.373683,0.499729,0.397483,1.469514,66.348452


In [48]:
get_result('CSMF', 5, 'tp', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_5/tp/slices_5/metrics/Final_result_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_5/tp/slices_5/metrics/Final_result_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_5/tp/slices_5/metrics/Final_result_density_tp_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_5/tp/slices_5/metrics/Final_result_density_tp_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_5/tp/slices_5/metrics/Final_result_density_tp_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_5/tp/slices_5/time/Final_time_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_5/tp/slices_5/time/Final_time_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_5/tp/slices_5/time/Final_time_

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,20.958466,60.796562,0.440528,0.369078,1.646592,24.181048
1,19.479916,57.210533,0.409254,0.339592,1.498240,31.996237
2,19.560162,57.009716,0.411310,0.341845,1.541927,36.734712
3,18.703306,55.131886,0.392968,0.326505,1.440915,41.390397
4,18.606686,54.893333,0.391300,0.317846,1.472284,54.952526


In [49]:
get_result('NeuCF', 5, 'tp', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_5/tp/slices_5/metrics/Final_result_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_5/tp/slices_5/metrics/Final_result_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_5/tp/slices_5/metrics/Final_result_density_tp_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_5/tp/slices_5/metrics/Final_result_density_tp_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_5/tp/slices_5/metrics/Final_result_density_tp_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_5/tp/slices_5/time/Final_time_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_5/tp/slices_5/time/Final_time_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_5/tp/slices_5/time/Fin

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,18.474222,56.700460,0.388379,0.331331,1.585582,30.324927
1,16.513690,51.389717,0.347219,0.301461,1.273231,47.744585
2,14.784818,47.450405,0.310873,0.260070,1.200062,74.156106
3,14.307692,46.167137,0.300831,0.246997,1.215671,83.594764
4,13.676521,44.662804,0.287667,0.230505,1.140903,99.827834


In [50]:
get_result('GraphMF', 5, 'tp', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_5/tp/slices_5/metrics/Final_result_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_5/tp/slices_5/metrics/Final_result_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_5/tp/slices_5/metrics/Final_result_density_tp_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_5/tp/slices_5/metrics/Final_result_density_tp_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_5/tp/slices_5/metrics/Final_result_density_tp_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_5/tp/slices_5/time/Final_time_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_5/tp/slices_5/time/Final_time_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_5/tp/s

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,13.636343,46.269670,0.286777,0.206896,1.036553,62.500106
1,12.760710,43.877766,0.268362,0.190771,0.928542,84.135306
2,12.083125,41.795300,0.254072,0.177583,0.886029,100.954954
3,11.651347,40.778008,0.244822,0.169132,0.815652,129.721314
4,11.328133,39.575294,0.238181,0.164718,0.826298,150.478770


In [51]:
get_result('MF', 5, 'tp', slices, 'all')

/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_5/tp/slices_5/metrics/Final_result_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_5/tp/slices_5/metrics/Final_result_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_5/tp/slices_5/metrics/Final_result_density_tp_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_5/tp/slices_5/metrics/Final_result_density_tp_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_5/tp/slices_5/metrics/Final_result_density_tp_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_5/tp/slices_5/time/Final_time_density_tp_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_5/tp/slices_5/time/Final_time_density_tp_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_5/tp/slices_5/time/Final_time_density_tp_0.20.

,MAE,RMSE,NMAE,MRE,NPRE,agg_time
0,24.277304,69.461700,0.510406,0.479199,2.447732,30.815330
1,21.245903,61.544533,0.446738,0.417988,1.989535,42.287986
2,20.115984,59.428123,0.422900,0.398512,1.929225,55.462856
3,18.984800,56.640205,0.399273,0.373688,1.714935,68.792578
4,19.548567,58.718190,0.410993,0.374618,1.608597,87.433689


*****

*****

# PURE SISA

In [52]:
get_result('CSMF', 6, 'rt', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [53]:
get_result('NeuCF', 6, 'rt', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [54]:
get_result('GraphMF', 6, 'rt', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [55]:
get_result('MF', 6, 'rt', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [56]:
get_result('CSMF', 6, 'tp', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [57]:
get_result('NeuCF', 6, 'tp', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [58]:
get_result('GraphMF', 6, 'tp', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [59]:
get_result('MF', 6, 'tp', slices, 'all')

Pure SISA experiment!
实验尚未进行


*****

In [60]:
get_result('CSMF', 7, 'rt', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [61]:
get_result('NeuCF', 7, 'rt', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [62]:
get_result('GraphMF', 7, 'rt', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [63]:
get_result('MF', 7, 'rt', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [64]:
get_result('CSMF', 7, 'tp', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [65]:
get_result('NeuCF', 7, 'tp', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [66]:
get_result('GraphMF', 7, 'tp', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [67]:
get_result('MF', 7, 'tp', slices, 'all')

Pure SISA experiment!
实验尚未进行


In [68]:
def final_get(string):

    rt = np.concatenate([np.array(get_result(string, 10, 'rt', 1, 'all'))[:, 0].reshape(-1, 1), np.array(get_result(string, 1, 'rt', slices, 'all'))[:, 0].reshape(-1, 1)], axis = 1)
    rt = np.concatenate([rt, np.array(get_result(string, 3, 'rt', slices, 'all'))[:, 0].reshape(-1, 1)], axis = 1)
    rt = np.concatenate([rt, np.array(get_result(string, 5, 'rt', slices, 'all'))[:, 0].reshape(-1, 1)], axis = 1)


    tp = np.concatenate([np.array(get_result(string, 10, 'tp', 1, 'all'))[:, 0].reshape(-1, 1), np.array(get_result(string, 1, 'tp', slices, 'all'))[:, 0].reshape(-1, 1)], axis = 1)
    tp = np.concatenate([tp, np.array(get_result(string, 3, 'tp', slices, 'all'))[:, 0].reshape(-1, 1)], axis = 1)
    tp = np.concatenate([tp, np.array(get_result(string, 5, 'tp', slices, 'all'))[:, 0].reshape(-1, 1)], axis = 1)

    ans = np.concatenate([rt, np.zeros((1, 4)), tp], axis = 0)
    ans = pd.DataFrame(ans)
    ans.columns = ['Retrain', 'SISA', 'Receraser', 'ours']
    ans.to_excel(f'./Result/{string}.xlsx', index = False)
    return ans

In [69]:
final_get('NeuCF')

/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/

,Retrain,SISA,Receraser,ours
0,0.392078,0.482301,0.476795,0.396871
1,0.363028,0.447009,0.445929,0.369182
2,0.346025,0.428050,0.421950,0.354376
3,0.333460,0.400578,0.401849,0.344369
4,0.322231,0.386466,0.393714,0.332609
5,0.000000,0.000000,0.000000,0.000000
6,16.512076,24.349040,22.936802,18.474222
7,15.105738,20.118294,19.949474,16.513690
8,13.949186,18.351050,18.478191,14.784818
9,13.243787,16.689625,17.519193,14.307692


In [70]:
final_get('CSMF')

/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/rt/slices_1/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/rt/slices_1/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/CSMF/part_type_10/rt/slices_1/time/Fin

,Retrain,SISA,Receraser,ours
0,0.381232,0.553634,0.524577,0.440785
1,0.352104,0.521528,0.511020,0.410742
2,0.332131,0.514238,0.494511,0.403900
3,0.321007,0.492257,0.476160,0.390911
4,0.311297,0.492931,0.461915,0.393640
5,0.000000,0.000000,0.000000,0.000000
6,13.971522,23.513325,22.771120,20.958466
7,12.828636,22.547432,22.141546,19.479916
8,11.991392,22.117840,21.075153,19.560162
9,11.371379,21.868866,20.459267,18.703306


In [71]:
final_get('MF')

/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/rt/slices_1/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/rt/slices_1/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/MF/part_type_10/rt/slices_1/time/Final_time_density_

,Retrain,SISA,Receraser,ours
0,0.421913,0.751829,0.746751,0.568700
1,0.371186,0.711287,0.736763,0.506450
2,0.354635,0.695839,0.727730,0.486147
3,0.345227,0.676129,0.702522,0.451137
4,0.338208,0.674864,0.689608,0.453907
5,0.000000,0.000000,0.000000,0.000000
6,16.426994,41.385647,41.523693,24.277304
7,14.493483,38.715672,38.711426,21.245903
8,13.636238,37.959404,35.515090,20.115984
9,13.125734,36.393730,33.154380,18.984800


In [72]:
final_get('GraphMF')

/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/rt/slices_1/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type_10/rt/slices_1/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/GraphMF/part_type

,Retrain,SISA,Receraser,ours
0,0.321518,0.346552,0.368496,0.341432
1,0.306773,0.328515,0.347934,0.324625
2,0.299721,0.318232,0.338666,0.312838
3,0.293260,0.309600,0.327587,0.306653
4,0.288732,0.305141,0.321771,0.301734
5,0.000000,0.000000,0.000000,0.000000
6,11.740687,13.676613,15.089726,13.636343
7,10.848979,12.634592,13.989352,12.760710
8,10.314256,11.937988,13.281751,12.083125
9,9.958744,11.394626,12.813992,11.651347


In [73]:
def final_get(string):

    rt_MAE = np.concatenate([np.array(get_result(string, 10, 'rt', 1, 'all'))[:, 0].reshape(-1, 1), np.array(get_result(string, 1, 'rt', slices, 'all'))[:, 0].reshape(-1, 1)], axis = 1)
    rt_MAE = np.concatenate([rt_MAE, np.array(get_result(string, 3, 'rt', slices, 'all'))[:, 0].reshape(-1, 1)], axis = 1)
    rt_MAE = np.concatenate([rt_MAE, np.array(get_result(string, 5, 'rt', slices, 'all'))[:, 0].reshape(-1, 1)], axis = 1)


    rt_NMAE = np.concatenate([np.array(get_result(string, 10, 'rt', 1, 'all'))[:, 2].reshape(-1, 1), np.array(get_result(string, 1, 'rt', slices, 'all'))[:, 2].reshape(-1, 1)], axis = 1)
    rt_NMAE = np.concatenate([rt_NMAE, np.array(get_result(string, 3, 'rt', slices, 'all'))[:, 2].reshape(-1, 1)], axis = 1)
    rt_NMAE = np.concatenate([rt_NMAE, np.array(get_result(string, 5, 'rt', slices, 'all'))[:, 2].reshape(-1, 1)], axis = 1)

    tp_MAE = np.concatenate([np.array(get_result(string, 10, 'tp', 1, 'all'))[:, 0].reshape(-1, 1), np.array(get_result(string, 1, 'tp', slices, 'all'))[:, 0].reshape(-1, 1)], axis = 1)
    tp_MAE = np.concatenate([tp_MAE, np.array(get_result(string, 3, 'tp', slices, 'all'))[:, 0].reshape(-1, 1)], axis = 1)
    tp_MAE = np.concatenate([tp_MAE, np.array(get_result(string, 5, 'tp', slices, 'all'))[:, 0].reshape(-1, 1)], axis = 1)


    tp_NMAE = np.concatenate([np.array(get_result(string, 10, 'tp', 1, 'all'))[:, 2].reshape(-1, 1), np.array(get_result(string, 1, 'tp', slices, 'all'))[:, 2].reshape(-1, 1)], axis = 1)
    tp_NMAE = np.concatenate([tp_NMAE, np.array(get_result(string, 3, 'tp', slices, 'all'))[:, 2].reshape(-1, 1)], axis = 1)
    tp_NMAE = np.concatenate([tp_NMAE, np.array(get_result(string, 5, 'tp', slices, 'all'))[:, 2].reshape(-1, 1)], axis = 1)

    ans = np.concatenate([rt_MAE, np.zeros((1, 4)), rt_NMAE, np.zeros((1, 4)), tp_MAE, np.zeros((1, 4)), tp_NMAE], axis = 0)
    ans = pd.DataFrame(ans)
    ans.columns = ['Retrain', 'SISA', 'Receraser', 'ours']
    ans.to_excel(f'./Result/{string}.xlsx', index = False)
    return ans

final_get('NeuCF')
final_get('CSMF')
final_get('MF')
final_get('GraphMF')

def cat_all_result():
    df1 = pd.read_excel('./Result/NeuCF.xlsx').to_numpy()
    df2 = pd.read_excel('./Result/CSMF.xlsx').to_numpy()
    df3 = pd.read_excel('./Result/MF.xlsx').to_numpy()
    df4 = pd.read_excel('./Result/GraphMF.xlsx').to_numpy()
    # print(df1.shape)
    ans = np.concatenate([df1, np.zeros((23, 2)), df2, np.zeros((23, 2)), df3, np.zeros((23, 2)), df4], axis = 1)
    ans = pd.DataFrame(ans)
    ans.columns = ['Retrain', 'SISA', 'Receraser', 'balance', '**', '**',
                   'Retrain', 'SISA', 'Receraser', 'balance', '**', '**',
                   'Retrain', 'SISA', 'Receraser', 'balance', '**', '**',
                   'Retrain', 'SISA', 'Receraser', 'balance', ]
    # ans.columns = ['Retrain', 'SISA','SISA2', 'Receraser', 'balance', 'non_balan', 'user_sim', 'item', '**', '**',
    #
    ans.index = ['0.10', '0.15', '0.20', '0.25', '0.30', '--', '0.10', '0.15', '0.20', '0.25', '0.30', '--', '0.10', '0.15', '0.20', '0.25', '0.30', '--', '0.10', '0.15', '0.20', '0.25', '0.30']
    ans.to_excel('MAE_NMAE.xlsx', index = True)
    return ans
cat_all_result()

/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.20.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.25.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/metrics/Final_result_density_rt_0.30.csv
----------------------------------------------------------------------------------------------------------------------------------
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/time/Final_time_density_rt_0.10.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/time/Final_time_density_rt_0.15.csv
/home/yxzeng/code_python/QoS_Eraser/Result/NeuCF/part_type_10/rt/slices_1/

,Retrain,SISA,Receraser,balance,**,**,Retrain,SISA,Receraser,balance,...,Retrain,SISA,Receraser,balance,**,**,Retrain,SISA,Receraser,balance
0.10,0.392078,0.482301,0.476795,0.396871,0.0,0.0,0.381232,0.553634,0.524577,0.440785,...,0.421913,0.751829,0.746751,0.568700,0.0,0.0,0.321518,0.346552,0.368496,0.341432
0.15,0.363028,0.447009,0.445929,0.369182,0.0,0.0,0.352104,0.521528,0.511020,0.410742,...,0.371186,0.711287,0.736763,0.506450,0.0,0.0,0.306773,0.328515,0.347934,0.324625
0.20,0.346025,0.428050,0.421950,0.354376,0.0,0.0,0.332131,0.514238,0.494511,0.403900,...,0.354635,0.695839,0.727730,0.486147,0.0,0.0,0.299721,0.318232,0.338666,0.312838
0.25,0.333460,0.400578,0.401849,0.344369,0.0,0.0,0.321007,0.492257,0.476160,0.390911,...,0.345227,0.676129,0.702522,0.451137,0.0,0.0,0.293260,0.309600,0.327587,0.306653
0.30,0.322231,0.386466,0.393714,0.332609,0.0,0.0,0.311297,0.492931,0.461915,0.393640,...,0.338208,0.674864,0.689608,0.453907,0.0,0.0,0.288732,0.305141,0.321771,0.301734
--,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
0.10,0.431424,0.530998,0.524875,0.436891,0.0,0.0,0.419521,0.609393,0.577429,0.485055,...,0.464470,0.827147,0.821947,0.626001,0.0,0.0,0.353885,0.381429,0.405631,0.375890
0.15,0.399488,0.492049,0.490766,0.406377,0.0,0.0,0.387357,0.573946,0.562791,0.451846,...,0.408538,0.782927,0.810518,0.557366,0.0,0.0,0.337634,0.361548,0.382844,0.357349
0.20,0.380859,0.471497,0.464453,0.390112,0.0,0.0,0.365400,0.565938,0.544128,0.444642,...,0.390518,0.765723,0.800783,0.534890,0.0,0.0,0.329798,0.350095,0.372464,0.344489
0.25,0.366833,0.441038,0.442376,0.378887,0.0,0.0,0.353164,0.541614,0.523767,0.430160,...,0.379931,0.744515,0.773922,0.496780,0.0,0.0,0.322822,0.340762,0.360766,0.337672
